In [0]:
emp = [(1,"Smith",-1,"2018","10","M",3000), \
    (2,"Rose",1,"2010","20","M",4000), \
    (3,"Williams",1,"2010","10","M",1000), \
    (4,"Jones",2,"2005","10","F",2000), \
    (5,"Brown",2,"2010","40","",-1), \
      (6,"Brown",2,"2010","50","",-1) \
  ]
empColumns = ["emp_id","name","superior_emp_id","year_joined", \
       "emp_dept_id","gender","salary"]



dept = [("Finance",10), \
    ("Marketing",20), \
    ("Sales",30), \
    ("IT",40) \
  ]
deptColumns = ["dept_name","dept_id"]


spark.createDataFrame(data=emp,schema=empColumns).createOrReplaceTempView('Employee')
spark.sql('select * from Employee').show()
spark.sql("select emp_id , name from Employee where emp_dept_id='10'").show()
spark.sql("select emp_id , name from Employee where name like 'S%'").show()
spark.sql("select emp_dept_id , count(*) as count from Employee group by emp_dept_id").show()
spark.sql("select emp_dept_id , count(*) as count from Employee group by emp_dept_id order by 2 desc limit 1").show()
spark.sql("select emp_id , name , emp_dept_id from Employee where salary=(select max(salary) from Employee)").show()
spark.sql("select emp_dept_id , count(*) as count from Employee group by emp_dept_id  having count(*) > 1 order by 2 desc limit 1").show()

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|
|     2|    Rose|              1|       2010|         20|     M|  4000|
|     3|Williams|              1|       2010|         10|     M|  1000|
|     4|   Jones|              2|       2005|         10|     F|  2000|
|     5|   Brown|              2|       2010|         40|      |    -1|
|     6|   Brown|              2|       2010|         50|      |    -1|
+------+--------+---------------+-----------+-----------+------+------+

+------+--------+
|emp_id|    name|
+------+--------+
|     1|   Smith|
|     3|Williams|
|     4|   Jones|
+------+--------+

+------+-----+
|emp_id| name|
+------+-----+
|     1|Smith|
+------+-----+

+-----------+-----+
|emp_dept_id|count|
+-----------+-----+
|         10|   

In [0]:

spark.createDataFrame(data=emp,schema=empColumns).createOrReplaceTempView('Employee')

spark.createDataFrame(data=dept,schema=deptColumns).createOrReplaceTempView('Department')
spark.sql('select * from Employee e inner join Department d on e.emp_dept_id = d.dept_id').show()
spark.sql('select * from Employee e left join Department d on e.emp_dept_id = d.dept_id').show()
spark.sql('select * from Employee e right join Department d on e.emp_dept_id = d.dept_id').show()
spark.sql('select * from Employee e full outer join Department d on e.emp_dept_id = d.dept_id').show()

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+-

In [0]:

simpleData = (("James", "Sales", 3000), \
    ("Michael", "Sales", 4600),  \
    ("Robert", "Sales", 4100),   \
    ("Maria", "Finance", 3000),  \
    ("James", "Sales", 3000),    \
    ("Scott", "Finance", 3300),  \
    ("Jen", "Finance", 3900),    \
    ("Jeff", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000),\
    ("Saif", "Sales", 4100) \
  )
 
empcolumns= ["employee_name", "department", "salary"]

spark.createDataFrame(data=simpleData,schema=empcolumns).createOrReplaceTempView('Employee')
spark.sql("select * , row_number() over(partition by department order by salary desc) as rn ,  rank() over(partition by department order by salary desc) as rank ,  dense_rank() over(partition by department order by salary desc) as dense_rn ,  lead(salary) over(partition by department order by salary desc) as lead ,  lag(salary) over(partition by department order by salary desc) as lag from Employee").show()

+-------------+----------+------+---+----+--------+----+----+
|employee_name|department|salary| rn|rank|dense_rn|lead| lag|
+-------------+----------+------+---+----+--------+----+----+
|          Jen|   Finance|  3900|  1|   1|       1|3300|null|
|        Scott|   Finance|  3300|  2|   2|       2|3000|3900|
|        Maria|   Finance|  3000|  3|   3|       3|null|3300|
|         Jeff| Marketing|  3000|  1|   1|       1|2000|null|
|        Kumar| Marketing|  2000|  2|   2|       2|null|3000|
|      Michael|     Sales|  4600|  1|   1|       1|4100|null|
|       Robert|     Sales|  4100|  2|   2|       2|4100|4600|
|         Saif|     Sales|  4100|  3|   2|       2|3000|4100|
|        James|     Sales|  3000|  4|   4|       3|3000|4100|
|        James|     Sales|  3000|  5|   4|       3|null|3000|
+-------------+----------+------+---+----+--------+----+----+

